In [1]:
import torch, transformers, torchaudio
from transformers import AutoFeatureExtractor, ASTForAudioClassification
import librosa

audio_path = 'sample.wav'
y, sr = librosa.load(audio_path, sr=None)

In [2]:
feature_extractor = AutoFeatureExtractor.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')
result = feature_extractor(y, return_tensors='pt')

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [3]:
model = ASTForAudioClassification.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')
prediction_logits = model(result['input_values']).logits

In [4]:
predicted_class_ids = torch.argmax(prediction_logits, dim=-1).item()
model.config.id2label[predicted_class_ids]

'Music'

In [5]:
from transformers import pipeline

pipe = pipeline('audio-classification', model='MIT/ast-finetuned-audioset-10-10-0.4593')
pipe.model

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTSdpaAttention(
            (attention): ASTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
       

In [6]:
pipe('sample.wav')

C:\code\audio\ai-audio\.venv\Lib\site-packages\transformers\models\audio_spectrogram_transformer\feature_extraction_audio_spectrogram_transformer.py:118: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  waveform = torch.from_numpy(waveform).unsqueeze(0)


[{'score': 0.6285784840583801, 'label': 'Music'},
 {'score': 0.05208112299442291, 'label': 'Theme music'},
 {'score': 0.031110258772969246, 'label': 'Video game music'},
 {'score': 0.027717722579836845, 'label': 'Musical instrument'},
 {'score': 0.020484384149312973, 'label': 'Background music'}]